In [1]:
import sys
import json
import cobra
import cplex
import re
import cobrakbase
import escher

from escher import Builder
from optlang.symbolics import Zero, add
#Put the path to ModelSEEDpy on your machine here
sys.path.append("/Users/ffoflonker/Documents/Command_line/ModelSEEDpy-csh")
#import modelseedpy.fbapkg
from modelseedpy import MSPackageManager
import pandas as pd
from sklearn.metrics import r2_score
import numpy as np

cobrakbase 0.2.7


In [2]:

res = np.geomspace(1e-8, 1e8, 17, endpoint = True)
res

array([1.e-08, 1.e-07, 1.e-06, 1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01,
       1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04, 1.e+05, 1.e+06, 1.e+07,
       1.e+08])

In [3]:
#Uncomment the tissue datastructure for the model you want to run
#Single tissue model:
tissues = ["Mature_leaf"]
#Two tissue model:
#tissues = ["Mature_leaf","GermEmbryo_2_DAI"]

#tissues = ["Mature_leaf","Zone_1","Embryo_20_DAP","Embryo_38_DAP","root_MZ", "root_EZ", "Endosperm_12_DAP"]
#16 tissue model:
#tissues = ["Mature_leaf","Zone_1","Zone_2","Zone_3","GermEmbryo_2_DAI","Pericarp_Aleurone_27_DAP","Endosperm_Crown_27_DAP","Endosperm_12_DAP","Endosperm_10_DAP","Endosperm_8_DAP","Embryo_38_DAP","Embryo_20_DAP","root_Stele","root_Cortex","root_EZ","root_MZ"]
kbase_api = cobrakbase.KBaseAPI()
#model = kbase_api.get_from_ws("C4_HS_Zmays_Model",25427)
model =cobra.io.read_sbml_model('C4_HS_Zmays_Model_v2.xml')
model.solver = 'optlang-cplex'
#Setting model to complete media
#media = kbase_api.get_from_ws("Carbon-D-Glucose","KBaseMedia")
pkgmgr = MSPackageManager.get_pkg_mgr(model,1)
pkgmgr.getpkg("KBaseMediaPkg",1).build_package(None)

#Reading reaction protein abundances
data = ""
with open('/Users/ffoflonker/Documents/Command_line/ModelSEEDpy-csh/examples/ReactionProtein.txt', 'r') as file:
    data = file.read()
lines = data.split("\n")
reaction_proteins = {}
headers = None
for line in lines:
    if headers == None:
        headers = line.split("\t")
    else:
        array = line.split("\t")
        for i in range(2, len(array)):
            if headers[i] not in reaction_proteins:
                reaction_proteins[headers[i]] = dict()
            reaction_proteins[headers[i]][array[0]] = abs(float(array[i]))

#Reading reaction kcat values
data = ""
with open('Kcats_grouped.txt', 'r') as file:
    data = file.read()
lines = data.split("\n")
reaction_kcat = {}
headers = None
for line in lines:
    if headers == None:
        headers = line.split("\t")
    else:
        array = line.split("\t")
        reaction_kcat[array[0]] = abs(float(array[1]))

#Reading measured reaction fluxes
data = ""
with open('measured_Claudia4_edit.txt', 'r') as file:
    data = file.read()
lines = data.split("\n")
reaction_measures = {}
headers = None
for line in lines:
    if headers == None:
        headers = line.split("\t")
    else:
        array = line.split("\t")
        if array[1] not in reaction_measures:
                reaction_measures[array[1]] = dict()
        rxnid = array[0]
        reaction_measures[array[1]][rxnid] = abs(float(array[2]))

biomass_fluxes = {
    "root_MZ" : 0.049925,
    "root_EZ" : 0.049925,
    "root_Cortex" : 0.049925,
    "root_Stele" : 0.049925,
    "Embryo_20_DAP" : 0.015591667,
    "Embryo_38_DAP" : 0.015591667,
    "Endosperm_8_DAP" : 0.010208333,
    "Endosperm_10_DAP" : 0.010208333,
    "Endosperm_12_DAP" : 0.010208333,
    "Endosperm_Crown_27_DAP" : 0.010208333,
    "Pericarp_Aleurone_27_DAP" : 0.001529167,
    "GermEmbryo_2_DAI" : 0.015591667,
    "Zone_1" : 0.006395833,
    "Zone_2" : 0.006395833,
    "Zone_3" : 0.006395833,
    "Mature_leaf" : 0.006395833
}

protein_fraction = {
	'root_MZ': 0.05,
	'root_EZ' :0.05,
	'root_Cortex': 0.05,
	'root_Stele' : 0.05,
	'Embryo_20_DAP' : 0.116,
	'Embryo_38_DAP' : 0.116,
	'Endosperm_8_DAP' : 0.08,
	'Endosperm_10_DAP' : 0.08,
	'Endosperm_12_DAP' : 0.08,
	'Endosperm_Crown_27_DAP' : 0.08,
	'Pericarp_Aleurone_27_DAP' : 0.09,
	'GermEmbryo_2_DAI' : 0.116,
	'Zone_1': 0.0948,
	'Zone_2': 0.0948,
	'Zone_3': 0.0948,
	'Mature_leaf': 0.0948
}

rxn = model.reactions.get_by_id("bio1_biomass")
rxn.upper_bound = 0.006395833
rxn.lower_bound = 0.006395833
sol=model.optimize()
# builder.model = model
# builder.reaction_data = sol.fluxes

modellist = []
count = 0
base_model = None
for tissue in tissues:
    print("Tissue:"+tissue)
    #Safe cloning method
    clone_model = cobra.io.json.from_json(cobra.io.json.to_json(model))
    #Applying flexible biomass constraints
    pkgmgr = MSPackageManager.get_pkg_mgr(clone_model,1)
    protein = protein_fraction[tissue]
    protein_flux = -1*(0.2-protein)
    pkgmgr.getpkg("FlexibleBiomassPkg",1).build_package({"bio_rxn_id":"bio1_biomass","use_rna_class":[-0.75,0.75],
        "use_dna_class":[-0.75,0.75],
        "use_protein_class":[protein_flux,protein_flux],
        "use_energy_class":[-0.1,0.1]})
    #Adding tissue specific proteome constraints
    package_params = {
        "rxn_proteome":reaction_proteins[tissue],
        #"target_flux":reaction_measures[tissue],
        "flux_values":reaction_measures[tissue],
        "prot_coef" : protein_fraction[tissue],#Set to the fraction of the cell that is protein
        "totalflux" : 1,#Set to one if we're fitting flux magnitude rather than actual flux to flux measurements
        "kcat_coef" : 0.02,#kapp = kcat_coef * kcat
        "set_objective":0
    }
    #Only base model fits kcat values
    if count == 0:
        package_params["kcat_values"] = reaction_kcat
    pkgmgr.getpkg("ProteomeFittingPkg",1).build_package(package_params)
    #Constraining biomass flux to reference value
    rxn = clone_model.reactions.get_by_id("bio1_biomass")
    rxn.upper_bound = biomass_fluxes[tissue]
    rxn.lower_bound = biomass_fluxes[tissue]
    if count == 0:
        base_model = clone_model
    else:
        modellist.append(clone_model)
    count += 1




Tissue:Mature_leaf


In [4]:
############Coefficient for kcat fitting:
#kcat_fit = 0.0001
###########Coefficient for flux fitting:
#flux_fit = 1
############Coefficient for kinetic fitting:
#kinetic_fit = 
results=[]
for x in res:
    kinetic_fit=x
    #kcat_fit=x
    for x in res:
        flux_fit=x
        for x in res:
            #kinetic_fit=x
            kcat_fit=x
            #Pulling based model package manager
            pkgmgr = MSPackageManager.get_pkg_mgr(base_model,1)
            #Merging all models together
            if count > 1:
                pkgmgr.getpkg("ProblemReplicationPkg",1).build_package({"models":modellist,"shared_variable_packages":{pkgmgr.getpkg("ProteomeFittingPkg"):["kapp"]}})
            #Creating combined objective function
            objvars = []



            #Adding base model kfit,kvfit,and vfit to objective
            for rxnid in pkgmgr.getpkg("FluxFittingPkg").variables["vfit"]:
                objvars.append(flux_fit * pkgmgr.getpkg("FluxFittingPkg").variables["vfit"][rxnid] ** 2)
            for rxnid in pkgmgr.getpkg("ProteomeFittingPkg").variables["kvfit"]: 
                objvars.append(kinetic_fit * pkgmgr.getpkg("ProteomeFittingPkg").variables["kvfit"][rxnid] ** 2)
            for rxnid in pkgmgr.getpkg("ProteomeFittingPkg").variables["kfit"]: 

                objvars.append(kcat_fit * pkgmgr.getpkg("ProteomeFittingPkg").variables["kfit"][rxnid] ** 2)

            #Adding sub model kvfit,and vfit to objective
            count = 0
            for submodel in modellist:
                submgr = MSPackageManager.get_pkg_mgr(submodel)
                for rxnid in submgr.getpkg("ProteomeFittingPkg").variables["kvfit"]:

                    varname = submgr.getpkg("ProteomeFittingPkg").variables["kvfit"][rxnid].name
                    objvars.append(kinetic_fit * pkgmgr.getpkg("ProblemReplicationPkg").variables[str(count)][varname] ** 2)
                for rxnid in submgr.getpkg("FluxFittingPkg").variables["vfit"]:
                    varname = submgr.getpkg("FluxFittingPkg").variables["vfit"][rxnid].name

                    objvars.append(flux_fit * pkgmgr.getpkg("ProblemReplicationPkg").variables[str(count)][varname] ** 2)
                count += 1
            #Setting the objective
            base_model.objective = base_model.problem.Objective(add(objvars), direction="min", sloppy=True)

            #Printing LP file
            with open('ProteomeDrivenModeling.lp', 'w') as out:
                out.write(str(base_model.solver))
            try:
                sol=base_model.optimize()
            except:
                print('error')
                continue
            # buildertwo.model = base_model
            # buildertwo.reaction_data = sol.fluxes
            #print ('done solving')
            
            #Pulling solution data
            solution_data = {}
            count = 0
            for tissue in tissues:
                solution_data[tissue] = {"flux":{},"vfit":{},"kfit":{},"kvfit":{},"kapp":{},"flex":{}}     
                if count == 0:
                    for reaction in base_model.reactions:
                        if re.search('FLEX_(cpd\d+_c0)', reaction.id) != None:
                            m = re.search('FLEX_(cpd\d+_c0)', reaction.id)
                            solution_data[tissue]["flex"][m[1]] = reaction.flux
                        elif re.search('([a-z]+)_flex', reaction.id) != None:
                            m = re.search('([a-z]+)_flex', reaction.id)
                            solution_data[tissue]["flex"][m[1]] = reaction.flux
                        else:
                            solution_data[tissue]["flux"][reaction.id] = reaction.flux
                    for rxnid in pkgmgr.getpkg("FluxFittingPkg").variables["vfit"]:
                        solution_data[tissue]["vfit"][rxnid] = pkgmgr.getpkg("FluxFittingPkg").variables["vfit"][rxnid].primal
                    for rxnid in pkgmgr.getpkg("ProteomeFittingPkg").variables["kfit"]:
                        solution_data[tissue]["kfit"][rxnid] = pkgmgr.getpkg("ProteomeFittingPkg").variables["kfit"][rxnid].primal
                    for rxnid in pkgmgr.getpkg("ProteomeFittingPkg").variables["kapp"]:

                        solution_data[tissue]["kapp"][rxnid] = pkgmgr.getpkg("ProteomeFittingPkg").variables["kapp"][rxnid].primal
                    for rxnid in pkgmgr.getpkg("ProteomeFittingPkg").variables["kvfit"]:
                        solution_data[tissue]["kvfit"][rxnid] = pkgmgr.getpkg("ProteomeFittingPkg").variables["kvfit"][rxnid].primal
                else:
                    submodel = modellist[count-1]
                    submgr = MSPackageManager.get_pkg_mgr(submodel)
                    for reaction in base_model.reactions:
                        if re.search('FLEX_(cpd\d+_c0)', reaction.id) != None:
                            m = re.search('FLEX_(cpd\d+_c0)', reaction.id)
                            solution_data[tissue]["flex"][m[1]] = pkgmgr.getpkg("ProblemReplicationPkg").variables[str(count-1)][reaction.forward_variable.name].primal
                            solution_data[tissue]["flex"][m[1]] += -1*pkgmgr.getpkg("ProblemReplicationPkg").variables[str(count-1)][reaction.reverse_variable.name].primal
                        elif re.search('([a-z]+)_flex', reaction.id) != None:
                            m = re.search('([a-z]+)_flex', reaction.id)
                            solution_data[tissue]["flex"][m[1]] = pkgmgr.getpkg("ProblemReplicationPkg").variables[str(count-1)][reaction.forward_variable.name].primal
                            solution_data[tissue]["flex"][m[1]] += -1*pkgmgr.getpkg("ProblemReplicationPkg").variables[str(count-1)][reaction.reverse_variable.name].primal
                        else:
                            solution_data[tissue]["flux"][reaction.id] = pkgmgr.getpkg("ProblemReplicationPkg").variables[str(count-1)][reaction.forward_variable.name].primal
                            solution_data[tissue]["flux"][reaction.id] += -1*pkgmgr.getpkg("ProblemReplicationPkg").variables[str(count-1)][reaction.reverse_variable.name].primal
                    for rxnid in submgr.getpkg("ProteomeFittingPkg").variables["kfit"]:
                        varname = submgr.getpkg("ProteomeFittingPkg").variables["kfit"][rxnid].name
                        solution_data[tissue]["kfit"][rxnid] = pkgmgr.getpkg("ProblemReplicationPkg").variables[str(count-1)][varname].primal
                    for rxnid in submgr.getpkg("ProteomeFittingPkg").variables["kvfit"]:
                        varname = submgr.getpkg("ProteomeFittingPkg").variables["kvfit"][rxnid].name
                        solution_data[tissue]["kvfit"][rxnid] = pkgmgr.getpkg("ProblemReplicationPkg").variables[str(count-1)][varname].primal
                    for rxnid in submgr.getpkg("FluxFittingPkg").variables["vfit"]:
                        varname = submgr.getpkg("FluxFittingPkg").variables["vfit"][rxnid].name
                        solution_data[tissue]["vfit"][rxnid] = pkgmgr.getpkg("ProblemReplicationPkg").variables[str(count-1)][varname].primal    
                count += 1
                
                
                ######## r2
                y_pred=[]
                y_true=[]
                for rxn in reaction_measures['Mature_leaf'].keys():
                    if rxn in model.reactions:
                        y_true.append(reaction_measures['Mature_leaf'][rxn])
                        y_pred.append(solution_data['Mature_leaf']['flux'][rxn])
                r2=r2_score(y_true,y_pred)
                results.append([kcat_fit,flux_fit,kinetic_fit,r2])
                print (kcat_fit,flux_fit,kinetic_fit,r2)
                          
                

1e-08 1e-08 1e-08 -0.1347731741665148
1e-07 1e-08 1e-08 -0.1347731741665148
1e-06 1e-08 1e-08 -0.1347731741665148
1e-05 1e-08 1e-08 -0.1347731741665148
0.0001 1e-08 1e-08 -0.1347731741665148
0.001 1e-08 1e-08 -0.13477317416651258
0.01 1e-08 1e-08 -0.1347731741665148
0.1 1e-08 1e-08 -0.1347731741665148
1.0 1e-08 1e-08 -0.1347731741665148
10.0 1e-08 1e-08 -0.1347731741665148
100.0 1e-08 1e-08 -0.1347731741665148
1000.0 1e-08 1e-08 -0.1347731741665148
10000.0 1e-08 1e-08 -0.1347731741665148
100000.0 1e-08 1e-08 -0.1347731741665148
1000000.0 1e-08 1e-08 -0.1347731741665148
10000000.0 1e-08 1e-08 -0.1347731741665148
100000000.0 1e-08 1e-08 -0.1347731741665148
1e-08 1e-07 1e-08 -0.12693153706886307
1e-07 1e-07 1e-08 -0.12693153706886307
1e-06 1e-07 1e-08 -0.12693153706886307
1e-05 1e-07 1e-08 -0.12693153706886307
0.0001 1e-07 1e-08 -0.12693153706886307
0.001 1e-07 1e-08 -0.12693153706886307
0.01 1e-07 1e-08 -0.12693153706886307
0.1 1e-07 1e-08 -0.12693153706886307
1.0 1e-07 1e-08 -0.12693153

/opt/anaconda3/lib/python3.7/site-packages/cobra/util/solver.py:416 UserWarning: solver status is 'infeasible'


1e-08 10000.0 1000000.0 -1.5055204551350072
1e-07 10000.0 1000000.0 -1.6984358103949333
1e-06 10000.0 1000000.0 -1.9117331165842963
1e-05 10000.0 1000000.0 -2.092475438528529
0.0001 10000.0 1000000.0 -2.285591000618293
0.001 10000.0 1000000.0 -2.4460023082534685
0.01 10000.0 1000000.0 -2.6123120311077717
0.1 10000.0 1000000.0 -2.791029723245877
1.0 10000.0 1000000.0 -3.027748147061325
10.0 10000.0 1000000.0 -3.18247114891658
100.0 10000.0 1000000.0 -3.2056982880945926
1000.0 10000.0 1000000.0 -3.208135560765797
10000.0 10000.0 1000000.0 -3.2083789412349013
100000.0 10000.0 1000000.0 -3.208404331520346
1000000.0 10000.0 1000000.0 -3.2084069194726723
10000000.0 10000.0 1000000.0 -3.2084071518872372
100000000.0 10000.0 1000000.0 -3.208407175720331


/opt/anaconda3/lib/python3.7/site-packages/cobra/util/solver.py:416 UserWarning: solver status is 'infeasible'


1e-08 100000.0 1000000.0 -0.21679967619145035
1e-07 100000.0 1000000.0 -0.25632935444823635
1e-06 100000.0 1000000.0 -0.285581826140046
1e-05 100000.0 1000000.0 -0.29648805160713065
0.0001 100000.0 1000000.0 -0.3432106291593182
0.001 100000.0 1000000.0 -0.4129494444244908
0.01 100000.0 1000000.0 -0.4383130980205243
0.1 100000.0 1000000.0 -0.4472413451119712
1.0 100000.0 1000000.0 -0.4453393465705726
10.0 100000.0 1000000.0 -0.4280405850675981
100.0 100000.0 1000000.0 -0.42637285348795717
1000.0 100000.0 1000000.0 -0.4262703255072453
10000.0 100000.0 1000000.0 -0.4262610203425601
100000.0 100000.0 1000000.0 -0.42626012610628083
1000000.0 100000.0 1000000.0 -0.42625999962294725
10000000.0 100000.0 1000000.0 -0.42626004089133596
100000000.0 100000.0 1000000.0 -0.4262600450191545
1e-08 1000000.0 1000000.0 -0.09269878780329077
1e-07 1000000.0 1000000.0 -0.090829536296815
1e-06 1000000.0 1000000.0 -0.09075093597053296
1e-05 1000000.0 1000000.0 -0.09345413042672135
0.0001 1000000.0 1000000.0 

...

1000000.0 100000000.0 100000000.0 -0.15895695307600444


/opt/anaconda3/lib/python3.7/site-packages/cobra/util/solver.py:416 UserWarning: solver status is 'infeasible'


10000000.0 100000000.0 100000000.0 -0.17373245359560352


/opt/anaconda3/lib/python3.7/site-packages/cobra/util/solver.py:416 UserWarning: solver status is 'infeasible'


100000000.0 100000000.0 100000000.0 -0.17521581740723224


In [6]:
df=pd.DataFrame(results)
df.to_csv("coef_opt.csv")

In [281]:

col=[]
for rxn in model.reactions:
    col.append(rxn.id)
sol_df=pd.DataFrame(col)
sol_df.set_index(0,inplace=True)
sol_df

""
0
rxn00001_c0
rxn00001_d0
rxn00001_g0
rxn00001_v0
rxn00001_w0
...
EX_cpd00013_e0
EX_cpd00001_e0
EX_cpd00075_e0


In [282]:

for tiss in tissues:
    df=pd.DataFrame.from_dict(solution_data[tiss])
    df.rename(columns={"flux": tiss}, inplace=True)
    df2=df[tiss]
    sol_df=sol_df.merge(df2, left_index=True, right_index=True)
    
sol_df

,Mature_leaf
rxn00001_c0,-9.215527e-03
rxn00001_d0,0.000000e+00
rxn00001_g0,0.000000e+00
rxn00001_v0,0.000000e+00
rxn00001_w0,0.000000e+00
...,...
EX_cpd00013_e0,1.217486e-04
EX_cpd00001_e0,1.734979e-03
EX_cpd00075_e0,7.502901e-04
DM_cpd02701_c0,1.598958e-07


In [283]:
sol_df.to_csv("Maize16tiss-results.csv", header=True)

In [16]:
reaction_measures['Mature_leaf']

0.031916667